In [6]:
'''
2) Check Open Alex, Crossref, (Arxiv) for pdfs
3) if unique add to pdf storage and add name to title column 
'''
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [2]:
import pathlib
from utils import topic_model_config

email = topic_model_config.EMAIL
db_path = pathlib.Path(topic_model_config.DATABASE_PATH)
all_pdf_path = pathlib.Path(topic_model_config.ALL_PDF_FOLDER_PATH)
selected_model_name = topic_model_config.MODEL

In [7]:
from database.database_manager import DatabaseManager
from publication_API.crossref_api import CrossrefAPI
from publication_API.arxiv_api import ArxivAPI
from publication_API.open_alex import OpenAlex



database_manager = DatabaseManager(db_path)
crossref_api = CrossrefAPI(email=email, rows=10, database_manager=database_manager)
open_alex_api = OpenAlex(database_manager=database_manager)
arxiv_api = ArxivAPI(rows=10, database_manager=database_manager)

author_list = database_manager.get_all_authors()

# API Calls to get PDF
![image info](/home/hudah/expert_field_project/data_pipeline/API_flow.png)

In [8]:
#Pull all publications
for author_id in author_list:
    arxiv_api.api_call(author_id)
    crossref_api.api_call(author_id)
    open_alex_api.api_call(author_id)
    

2024-12-09 12:17:31,733 - INFO - Searching for publications using Arxiv started...


2024-12-09 12:17:32,455 - INFO - PDF already exists at /nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/aekowals/1908.05810v2.pdf, skipping download.
2024-12-09 12:17:32,478 - INFO - PDF already exists at /nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/aekowals/1908.05811v2.pdf, skipping download.
2024-12-09 12:17:32,526 - INFO - Downloading PDF from http://arxiv.org/pdf/1912.06739v4 to /nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/aekowals/1912.06739v4.pdf
2024-12-09 12:17:32,569 - INFO - Successfully downloaded PDF to /nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/aekowals/1912.06739v4.pdf
2024-12-09 12:17:32,577 - INFO - Likley a Duplicate : Counting Defiers: Examples from Health Care
2024-12-09 12:17:35,703 - INFO - Downloading PDF from http://arxiv.org/pdf/hep-ph/0112083v1 to /nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/aekowals/0112083v1.pdf
2024-12-09 12:17:35,709 - INFO - Successfully downloaded PDF

KeyboardInterrupt: 

In [9]:
from NLP.data_processor import TopicModelDataPreprocessor
from NLP.hdp import HDPTopicModel
import NLP.BERT
# from NLP.fastopic import FASTopicModel
from NLP.lda import LDATopicModel
from NLP.zeroshot import ZeroShotClassifier
# from NLP.guidedlda import GuidedLDATopicModel


#huesmann
topic_model_processor = TopicModelDataPreprocessor()

match selected_model_name:
    case "hdp":
        for author_folder in all_pdf_path.iterdir():
            if author_folder.is_dir():
                docs = topic_model_processor.get_and_process_pdf_files(author_folder=author_folder)
                model_obj = HDPTopicModel()
                model_obj.train(docs)
                topics = model_obj.get_topics()
                topics = model_obj.log_topics(topics)
    case "lda":
        for author_folder in all_pdf_path.iterdir():
            if author_folder.is_dir():
                docs = topic_model_processor.get_and_process_pdf_files(author_folder=author_folder)
                model_obj = LDATopicModel()
                lda_model, corpus, dictionary = model_obj.train(docs)
                topics = model_obj.get_topics(lda_model=lda_model)
    case "zeroshot":
        for author_folder in all_pdf_path.iterdir():
            if author_folder.is_dir():
                docs = topic_model_processor.get_entire_author_text(author_folder=author_folder)
                model_obj = ZeroShotClassifier()
                results_df = model_obj.classify_with_confidence_threshold(docs)
                quality_analysis = model_obj.analyze_classification_quality(results_df)
                model_obj.log_topics(results_df, quality_analysis)
    case "bert":
        NLP.BERT.main()
    case "fastopic":
        # model = FASTopicModel()
        pass
    # case "guidedlda":
    #     model = GuidedLDATopicModel()
    #     def get_topics(model, docs):
    #         model.initialize(docs)
    #         return model.topic_distribution(top_n=10)
    case _:
        raise ValueError(f"Unsupported topic model: {selected_model_name}")

# Process documents and retrieve topics




[nltk_data] Downloading package stopwords to /home/hudah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-12-09 12:19:28,200 - WARNING - couldn't open /nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/jdos/xml.pdf
2024-12-09 12:19:28,205 - WARNING - couldn't open /nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/jdos/589297.pdf
2024-12-09 12:19:29,964 - INFO - collecting all words and their counts
2024-12-09 12:19:29,965 - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-12-09 12:19:29,990 - INFO - collected 34494 token types (unigram + bigrams) from a corpus of 34635 words and 8 sentences
2024-12-09 12:19:29,992 - INFO - merged Phrases<34494 vocab, min_count=2, threshold=10, max_vocab_size=40000000>
2024-12-09 12:19:29,992 - INFO - Phrases lifecycle event {'msg': 'built Phrases<34494 vocab, min_count=2, threshold=10, max_vocab_size=40000000> in 0.03s', 'datetime': '2024-12-09T12:19:29.992694', 'gensim': '4.3.3',

KeyboardInterrupt: 

In [5]:
import NLP.BERT

match "bert":
    case "bert":
        NLP.BERT.main()
    case _:
        raise ValueError(f"Unsupported topic model: {selected_model_name}")

# Process documents and retrieve topics




[nltk_data] Downloading package wordnet to /home/hudah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/hudah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hudah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-12-09 12:12:07,650 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-12-09 12:12:08,259 - INFO - Use pytorch device_name: cuda
2024-12-09 12:12:14,655 - ERROR - Error processing xml.pdf: Failed to open file '/nfs/turbo/si-acastel/expert_field_project/full_pdfs_by_author/huesmann/xml.pdf'.
2024-12-09 12:12:18,415 - INFO - Running BERT on huesmann
2024-12-09 12:12:18,415 - INFO - Number of documents: 75


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-12-09 12:12:19,505 - INFO - Embedding shape: (75, 384)
2024-12-09 12:12:19,506 - INFO - UMAP configured with n_neighbors=15, n_components=5
2024-12-09 12:12:19,509 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2024-12-09 12:12:20,226 - INFO - Use pytorch device_name: cuda
2024-12-09 12:12:25,971 - INFO - Topics for huesmann:    Topic  Count                                  Name  \
0      0     63  0_child_aggression_violence_behavior   
1      1     12   1_reserved_collection_right_science   

                                      Representation  \
0  [child, aggression, violence, behavior, aggres...   
1  [reserved, collection, right, science, social,...   

                                 Representative_Docs  
0  [psychological science public interest decembe...  
1     [, , social science collection right reserved]  


# Zeroshot

In [4]:
from NLP.data_processor import TopicModelDataPreprocessor
from NLP.zeroshot import ZeroShotClassifier
import pathlib


author_folder = pathlib.Path('/home/hudah/expert_field_project/full_pdfs_by_author/huesmann/')
topic_model_processor = TopicModelDataPreprocessor()


match "zeroshot":
    case "zeroshot":
            docs = topic_model_processor.get_entire_author_text(author_folder=author_folder)
            model_obj = ZeroShotClassifier()
            results_df = model_obj.classify_with_confidence_threshold(docs)
            quality_analysis = model_obj.analyze_classification_quality(results_df)
            model_obj.log_topics(results_df, quality_analysis)
    case _:
        raise ValueError(f"Unsupported topic model: {selected_model_name}")

# Process documents and retrieve topics




2024-12-09 12:08:34,407 - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-12-09 12:08:34,408 - INFO - NumExpr defaulting to 8 threads.
2024-12-09 12:08:35,889 - WARNING - couldn't open /home/hudah/expert_field_project/full_pdfs_by_author/huesmann/xml.pdf
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
2024-12-09 12:12:02,234 - INFO - 
Enhanced Zero-Shot Classification Results:
2024-12-09 12:12:02,240 - INFO -                                                                                                document predicted_topic confidence alternative_topics
 author manuscript this article is protected by copyright all rights reserved risk factors for youth...        Robotics     12.82%  Business (12.52%)
 author manuscript this article is protected by copyright all rights reserved risk factors for youth...        Robotics     12.8

# HDP

In [1]:
from NLP.data_processor import TopicModelDataPreprocessor
import pathlib
from NLP.hdp import HDPTopicModel

author_folder = pathlib.Path('/home/hudah/expert_field_project/full_pdfs_by_author/huesmann/')
topic_model_processor = TopicModelDataPreprocessor()

match 'hdp':
    case "hdp":
        docs = topic_model_processor.get_and_process_pdf_files(author_folder=author_folder)
        model_obj = HDPTopicModel()
        model_obj.train(docs)
        topics = model_obj.get_topics()
        topics = model_obj.log_topics(topics)
    case _:
        raise ValueError(f"Unsupported topic model: {selected_model_name}")

# Process documents and retrieve topics


2024-12-09 11:59:36,458 - WARNING - couldn't open /home/hudah/expert_field_project/full_pdfs_by_author/huesmann/xml.pdf
2024-12-09 11:59:53,290 - INFO - collecting all words and their counts
2024-12-09 11:59:53,292 - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-12-09 11:59:53,518 - INFO - collected 168754 token types (unigram + bigrams) from a corpus of 298231 words and 75 sentences
2024-12-09 11:59:53,519 - INFO - merged Phrases<168754 vocab, min_count=2, threshold=10, max_vocab_size=40000000>
2024-12-09 11:59:53,519 - INFO - Phrases lifecycle event {'msg': 'built Phrases<168754 vocab, min_count=2, threshold=10, max_vocab_size=40000000> in 0.23s', 'datetime': '2024-12-09T11:59:53.519948', 'gensim': '4.3.3', 'python': '3.11.10 (main, Oct  3 2024, 07:29:13) [GCC 11.2.0]', 'platform': 'Linux-4.18.0-477.51.1.el8_8.x86_64-x86_64-with-glibc2.28', 'event': 'created'}
2024-12-09 11:59:53,520 - INFO - exporting phrases from Phrases<168754 vocab, min_count=2, thresho

# LDA

In [ ]:
import pathlib
from NLP.lda import LDATopicModel


author_folder = pathlib.Path('/home/hudah/expert_field_project/full_pdfs_by_author/huesmann/')
topic_model_processor = TopicModelDataPreprocessor()

match "lda":
    case "lda":
            docs = topic_model_processor.get_and_process_pdf_files(author_folder=author_folder)
            model_obj = LDATopicModel()
            lda_model, corpus, dictionary = model_obj.train(docs)
            topics = model_obj.get_topics(lda_model=lda_model)
    case _:
        raise ValueError(f"Unsupported topic model: {selected_model_name}")

# Process documents and retrieve topics




2024-12-09 11:20:04,560 - WARNING - couldn't open /home/hudah/expert_field_project/full_pdfs_by_author/huesmann/xml.pdf
2024-12-09 11:20:18,525 - INFO - collecting all words and their counts
2024-12-09 11:20:18,526 - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-12-09 11:20:18,753 - INFO - collected 168754 token types (unigram + bigrams) from a corpus of 298231 words and 75 sentences
2024-12-09 11:20:18,753 - INFO - merged Phrases<168754 vocab, min_count=2, threshold=10, max_vocab_size=40000000>
2024-12-09 11:20:18,754 - INFO - Phrases lifecycle event {'msg': 'built Phrases<168754 vocab, min_count=2, threshold=10, max_vocab_size=40000000> in 0.23s', 'datetime': '2024-12-09T11:20:18.754314', 'gensim': '4.3.3', 'python': '3.11.10 (main, Oct  3 2024, 07:29:13) [GCC 11.2.0]', 'platform': 'Linux-4.18.0-477.51.1.el8_8.x86_64-x86_64-with-glibc2.28', 'event': 'created'}
2024-12-09 11:20:18,755 - INFO - exporting phrases from Phrases<168754 vocab, min_count=2, thresho